# Learning and Decision Making

## Laboratory 5: Supervised learning

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. The IRIS dataset

The Iris flower data set is a data set describing the morphologic variation of Iris flowers of three related species. Two of the three species were collected in the Gaspé Peninsula "all from the same pasture, and picked on the same day and measured at the same time by the same person with the same apparatus".

The data set consists of 50 samples from each of three species of Iris (_Iris setosa_, _Iris virginica_ and _Iris versicolor_). Four features were measured from each sample: the length and the width of the sepals and petals, in centimetres.

In your work, you will use only two of the four features and consider only two of the three species of Iris.

---

We start by loading the dataset and plotting the two classes of points that we wish to discriminate.

In [0]:
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn import datasets

# Load dataset
iris = datasets.load_iris()
#print(iris.DESCR)

X = iris.data[50:,2:]
a = iris.target[50:]

# Plot data
plt.plot(X[:50, 0], X[:50, 1], 'bx', label='Versicolour')
plt.plot(X[50:, 0], X[50:, 1], 'ro', label='Virginica')
plt.xlabel('Petal length (cm)')
plt.ylabel('Petal width (cm)')
plt.legend(loc='best')
plt.show()

---

#### Activity 1.        

Train a logistic regression classifier in Python using Newton-Raphson's method. The method is described by the update:

$$\mathbf{w}^{(k+1)}\leftarrow\mathbf{w}^{(k)}-\mathbf{H}^{-1}\mathbf{g},$$

where $\mathbf{H}$ and $\mathbf{g}$ are the _Hessian matrix_ and _gradient vector_ that you computed in your homework. Therefore, to train the classifier you should write a cycle that repeatedly updates the parameter vector according to the rule above until the difference between two iterations is sufficiently small (e.g., smaller than $10^{-5}$).

Print the resulting parameters and plot the decision boundary over the data points. Make sure that:

1. You augment your data pointa with an extra coordinate that is always 1
2. The output vector takes only values 0 and 1
3. You initialize your parameters to zero.

**Note:** Don't forget to import `numpy`.

---

In [0]:
import numpy as np
import math
from matplotlib.colors import ListedColormap

def calc_new_pi(w, x):
    return 1 / (1 + np.exp(- np.dot(x, w)))

def calc_new_gradient(x, a, p):
    return np.dot((a - p[:,0]), x)

def calc_new_hessian(pi_arg):
    return -Xn.T.dot(np.diag(pi_arg[:,0] * (1 - pi_arg[:,0])).dot(Xn))

def calc_new_parameters(w_old):
    pi = calc_new_pi(w_old, Xn)
    h = calc_new_hessian(pi)
    g = calc_new_gradient(Xn, an, pi)
    w_new = w_old - np.linalg.inv(h).dot(g[:, None])
    return pi, h, g, w_new

def assert_different(w_o, w_n):
    err = pow(10, -5)
    distance = np.linalg.norm(w_o - w_n)
    return distance >= err

an = a - 1
Xn = np.append(X, np.ones((100,1)),  axis = 1)
#print(Xn)
w = np.zeros((3,1))
pi = calc_new_pi(w, Xn)
g = calc_new_gradient(Xn, an, pi)
#print("\n\n Gradient:",g)
h = calc_new_hessian(pi)
#print("\n\n Hessian: ",h)

w_old = w
w_new = w - np.linalg.inv(h).dot(g[:, None])
iterations = 0
while(assert_different(w_old, w_new)):
    w_old = w_new
    pi, h, g, w_new = calc_new_parameters(w_old)
    #print(w_new)

w = w_new

print("pi: \n", pi)
print("\ngradient: \n", g)
print("\nhessian: \n", h)
print("\nw: \n", w)
    
Xn = Xn[:,:2]

condition0 = pi < 0.5
X0i = np.extract(condition0, np.arange(100))
X0 = Xn[X0i, :]
condition1 = np.invert(condition0)
X1i = np.extract(condition1, np.arange(100))
X1 = Xn[X1i, :]
#Xlabel = condition1.astype(int)

#getting the decision boundary: y = mx + b
#for x = 0
x1 = 0
y1 = -w[2]/w[1]
#for y = 0
x2 = -w[2]/w[0]
y2 = 0
m = (y1 - y2)/(x1 - x2)
b = y1

xValues = np.arange(2.5, 7.5, 0.5)
yValues = m*xValues + b

plt.figure()
plt.plot(X0[:, 0], X0[:, 1], 'bx', label='Versicolour')
plt.plot(X1[:, 0], X1[:, 1], 'ro', label='Virginica')
plt.plot(xValues, yValues)
plt.xlabel('Petal length (cm)')
plt.ylabel('Petal width (cm)')
plt.legend(loc='best')
plt.show()

---

#### Activity 2.        

Compare your classifier from Activity 1 with the logistic regression classifier implemented `sci-kit learn`. The code block below already loads and constructs a logistic regression model. 

To compare you must first fit the model to the data from Activity 1 (use the method `fit`). Next, you should build a fine grid of points $(x, y)$ in feature space (try using the `numpy` function `meshgrid`) and compute the corresponding class using the classifier (use the method `predict`). You can then use the function `plt.pcolormesh` to plot the resulting regions of decision.

---

In [0]:
from sklearn.linear_model import LogisticRegression
from matplotlib.colors import ListedColormap

model = LogisticRegression(solver='newton-cg', C=1e40)
model.fit(Xn, an)

meshX, meshY = np.meshgrid(np.arange(2.8, 7.2, 0.005), np.arange(0.8, 2.8, 0.005))

meshData = np.c_[meshX.ravel(), meshY.ravel()]
meshPred = model.predict(meshData)
numPoints = meshPred.shape[0]

plt.figure()
plt.plot(X0[:, 0], X0[:, 1], 'bx', label='Versicolour')
plt.plot(X1[:, 0], X1[:, 1], 'ro', label='Virginica')
cmap_light = ListedColormap(['#AAAAFF', '#FFAAAA'])
plt.pcolormesh(meshX, meshY, meshPred.reshape(meshX.shape[0],meshX.shape[1]), cmap=cmap_light)
plt.show()

### 2. SPAM filtering

You will now implement a spam filter, in which you will compare the results of different classifiers seen in class. In order to do so, you will first need to prepare the data for learning.

The following block of code illustrates how you can use the `os` module to access a list of files in a given folder. In particular, if you uncompress the file `data.zip` your working folder, the instruction `os.listdir('data')` will return a list with the contents of folder `data`.

In [0]:
import os

print(os.listdir('data'))

Uncompress the data file `data.zip` to your current folder. You will find a total of four folders, named `spam-train`, `nonspam-train`, `spam-test` and `nonsmap-test`. Each folder contains a number of text files which have been pre-processed to remove stop-words, punctuation signs, and other non-informative elements. 

---

#### Activity 3.        

You will now select the 3,000 most frequent words appearing in the training data. You will use the number of occurrences of these words in each e-mail as the features that describe that e-mail. The code provided already goes over all the files in the folders `*-train` and builds a dictionary (actually, a `Counter`) containing all words appearing and how often they appear. 

Use the information in the (`Counter`) dictionary to select the 3,000 most frequent words. Before compiling the list of most common words, make sure to remove _non-words_---for which you can use the `isalpha` method of the `str` class---and _words of length 1_. To build the list of most frequent words, you may find useful the method `most_common` of the Counter class. Make sure you end up with a _sorted list_ containing the 3,000 most frequent words. 

---

In [0]:
from collections import Counter

words = []

files = []
for training_dir in ['data\\spam-train', 'data\\nonspam-train']:
    files += [os.path.join(training_dir, f) for f in os.listdir(training_dir)]

for f in files:
    fin = open(f, 'r')
    words += str(fin.read()).split()        
    fin.close()

d = Counter(words)


# Filter non words and words smaller than 1 char long
d = {k: v for k, v in d.items() if k.isalpha() and len(k)>1}
# Filter 3000 most common
d = Counter(d).most_common(3000)

Each of the files in the folder `spam-train` corresponds to a datapoint $(\mathbf{x}_n,a_n)$, where $\mathbf{x}_n$ is a vector containing the number of times that each of the most frequent words (computed in Activity 3) appears in that file, and $a_n$ is $0$. Conversely, each of the files in the folder `nonspam-train` corresponds to a datapoint $(\mathbf{x}_n,a_n)$, where $\mathbf{x}_n$ is again a vector containing the number of times that each of the most frequent words appears in that file, and $a_n$ is $1$. 

---

#### Activity 4.        

Go over the files in the aforementioned folders and create the a matrix `X` where each row $i$ is the datapoint corresponding to file $i$, and each column $j$ contains the number of times that the word $j$ appears in each of the files. Create the corresponding vector `a` of labels, where the component $i$ is 0 or 1 depending on whether file $i$ is spam or not.

** Note: ** You may want to create a function that receives the name of a folder and a list of words as arguments and returns the matrix of datapoints corresponding to the files in that folder, where each datapoint is described as a vector of features and each feature corresponds to the number of occurrences of the words in the list provided.

** Note 2: ** Extracting the features corresponding to the files may take a bit, so don't despair.

---

In [0]:
#vector with the 3000 most common words
wordVec = []
for elem in d:
    wordVec.append(elem[0])

X = np.zeros((len(files), len(wordVec)))

#auxiliary
fileWords = []
print("Computing X...")
for i in range(len(files)):
    fin = open(files[i], 'r')
    fileWords += str(fin.read()).split()
    
    #dict with all the valid words
    wordCounter = Counter(fileWords)
    wordCounter = {k: v for k, v in wordCounter.items() if k.isalpha() and len(k)>1}
    
    #remove any word not in wordVec
    toDelete = []
    for k, v in wordCounter.items():
        if k not in wordVec:
            toDelete.append(k)
    for w in toDelete:
        wordCounter.pop(w)
    
    #filling each row
    for j in range(len(wordVec)):
        toDelete = ""
        for k, v in wordCounter.items():
            if k == wordVec[j]:
                X[i, j] = v
                toDelete = k
                break
        
        #just to make things faster we delete the used keys
        if (toDelete != ""):
            wordCounter.pop(toDelete)
    
    fileWords = []
    fin.close()
    if (i % 100 == 0):
        print(i, "/", len(files))

print("X is computed")

#a vector
for i in range(len(files)):
    if (files[i].split("\\")[1] != 'spam-train'):
        break
a = np.append(np.zeros(i), np.ones(len(files) - i), axis = 0)

#this proves that it's working!
#i = 0
#test = np.zeros(len(d))
#for k, v in d:
#    test[i] = v
#    i = i + 1
#print("\n",np.array_equal(test, np.sum(X, axis = 0)))

print("\nX matrix:\n", X)
print("\na vector:\n", a)

Now that you have compiled your training set, you will train three different classifiers with the same dataset: a discriminant function (SVM), a discriminative model (LR) and a generative model (NB), and compare the performance of all three in terms of training time and performance on the test set. In order to do that, you must import the three classifiers and train them, much like you did with the LR classifier in Activity 2. 

The three classifiers have already been constructed for you.

---

#### Activity 5.

Train the three classifiers with the data that you collected in Activity 4. Report the time that each classifier took to train.

---

In [0]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import time

# SVM model
svm_model = LinearSVC()

# Logistic regression model
lr_model = LogisticRegression()

# Naive Bayes model
nb_model = MultinomialNB()

t0 = time.time()
svm_model.fit(X, a)
svmT = time.time() - t0

t0 = time.time()
lr_model.fit(X, a)
lrT = time.time() - t0

t0 = time.time()
nb_model.fit(X, a)
nbT = time.time() - t0

print("SVM model time: ", svmT)
print("\nLogistic Regression model time: ", lrT)
print("\nNaive Bays model time: ", nbT)

Finally, you will test the performance of the three classifiers in the test data. To that purpose, you must read the data in the `*-test` folders into a matrix of test points and the corresponding labels, and compare your predictions in this data with the actual labels. 

---

#### Activity 6.

For the messages in the folders `*-test` compute the predictions of your classifiers. Then, use the function `confusion_matrix` (which has been imported for you) to analyze the performance of your method. Report the accuracy of each classifier (i.e., the percentage of correct answers) and comment on the advantages and disadvantages of the three methods for this task.

---

In [0]:
from sklearn.metrics import confusion_matrix

words = []

files = []
for training_dir in ['data\\spam-test', 'data\\nonspam-test']:
    files += [os.path.join(training_dir, f) for f in os.listdir(training_dir)]

X = np.zeros((len(files), len(wordVec)))

#auxiliary
print("Computing X...")
fileWords = []
for i in range(len(files)):
    fin = open(files[i], 'r')
    fileWords += str(fin.read()).split()
    
    #dict with all the valid words
    wordCounter = Counter(fileWords)
    wordCounter = {k: v for k, v in wordCounter.items() if k.isalpha() and len(k)>1}
    
    #remove any word not in wordVec
    toDelete = []
    for k, v in wordCounter.items():
        if k not in wordVec:
            toDelete.append(k)
    for w in toDelete:
        wordCounter.pop(w)
    
    #filling each row
    for j in range(len(wordVec)):
        toDelete = ""
        for k, v in wordCounter.items():
            if k == wordVec[j]:
                X[i, j] = v
                toDelete = k
                break
        
        #just to make things faster we delete the used keys
        if (toDelete != ""):
            wordCounter.pop(toDelete)
    
    fileWords = []
    fin.close()
    if (i % 100 == 0):
        print(i, "/", len(files))

print("X is computed\n")

#a vector
for i in range(len(files)):
    if (files[i].split("\\")[1] != 'spam-test'):
        break
a = np.append(np.zeros(i), np.ones(len(files) - i), axis = 0)

t0 = time.time()
svmPredict = svm_model.predict(X)
svmPredictT = time.time() - t0

t0 = time.time()
lrPredict = lr_model.predict(X)
lrPredictT = time.time() - t0

t0 = time.time()
nbPredict = nb_model.predict(X)
nbPredictT = time.time() - t0

numFiles = a.shape[0]



cm = confusion_matrix(a, svmPredict)
svmHits = cm[0,0] + cm[1,1]
cm = confusion_matrix(a, lrPredict)
lrHits = cm[0,0] + cm[1,1]
cm = confusion_matrix(a, nbPredict)
nbHits = cm[0,0] + cm[1,1]

print("Analysis:\n")
print("SVM fit time: ", svmT)
print("SVM predict time: ", svmPredictT)
print("SVM total time: ", svmT + svmPredictT)
print("SVM performance: ", svmHits,"/", numFiles, "(", np.round(100*svmHits/numFiles, 2), "%)")

print("\n------\n")
print("LR fit time: ", lrT)
print("LR predict time: ", lrPredictT)
print("LR total time: ", lrT + lrPredictT)
print("LR performance: ", lrHits,"/", numFiles, "(", np.round(100*lrHits/numFiles, 2), "%)")

print("\n------\n")
print("NB fit time: ", nbT)
print("NB predict time: ", nbPredictT)
print("NB total time: ", nbT + nbPredictT)
print("NB performance: ", nbHits,"/", numFiles, "(", np.round(100*nbHits/numFiles, 2), "%)")

In [0]:
print("In terms of performance, NB is the worse of the three, missing 5 classifications. Both SVM and LR only fail 3.")
print("We divided the time analysis in two parts, the time needed to fit the data and the time needed to predict.")
print("NB is the model that takes less time to fit the data (by a big margin), followed by SVM and LR.")
print("LR is the model that takes less time to predict, followed by NB and SVM.")
print("NB takes significant less total time than the other two models. SVM is ", np.round(lrT + lrPredictT - svmT + svmPredictT, 3), "s faster than LR.")